# Logistic regression on Criteo dataset without Spark MLlib

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!curl -O https://dlcdn.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()
ss = spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import types as st
from pyspark.sql import functions as sf
from pyspark.sql import Row, DataFrame
from pyspark import RDD
from pyspark import StorageLevel

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plot
from typing import Tuple, Dict

## Load dataset

In [ ]:
integer_features = [f"int_feat_{i}" for i in range(1, 14)]
categorical_features = [f"cat_feat_{i}" for i in range(1, 27)]

fields = []

fields.append(
    st.StructField("label", st.IntegerType(), nullable=False)
)

for int_feat in integer_features:
    fields.append(st.StructField(int_feat, st.IntegerType(), nullable=True))
    
for cat_feat in categorical_features:
    fields.append(st.StructField(cat_feat, st.StringType(), nullable=True))

schema = st.StructType(fields)

In [ ]:
from urllib import request
toy_dataset_url = 'https://www.dropbox.com/s/hn2pt29gbne9c99/criteo_toy_dataset-part2.txt?dl=1'
request.urlretrieve(toy_dataset_url, "train.txt")

In [ ]:
full_df = ss.read.csv(
    path="train.txt",
    sep="\t",
    header=None,
    schema=schema
)

In [ ]:
full_df.agg(
    sf.count('*').alias('num_examples'),
    sf.sum('label').alias('num_positives'),
    sf.sum(sf.expr('label == 0').cast('int')).alias('num_negatives'),
    sf.mean('label').alias('avg_label')
).toPandas()

## Convert to vector with one hot encoding

### Select subset of features based on number of modalities

In [ ]:
threshold = 100
num_modalities = {} 
for cat_feat in categorical_features:
    num_modalities[cat_feat] = full_df \
        .filter(sf.col(cat_feat).isNotNull()) \
        .groupby(cat_feat) \
        .count() \
        .filter(sf.col('count') > sf.lit(threshold)) \
        .count()
num_modalities

In [ ]:
low_card_cat_feat = [cat_feat for cat_feat, num_modalities in num_modalities.items() if num_modalities < 50]
low_card_cat_feat

### Build map for one hot encoding

In [ ]:
modalities = {}
for cat_feat in low_card_cat_feat:
    rows = full_df\
        .filter(sf.col(cat_feat).isNotNull())\
        .groupby(cat_feat)\
        .count()\
        .filter(sf.col('count') > sf.lit(threshold))\
        .select(cat_feat)\
        .collect()
    modalities[cat_feat] = [row[cat_feat] for row in rows]

In [ ]:
one_hot_encoder = {cat_feat:{} for cat_feat in low_card_cat_feat}
#one_hot_encoder = [cat_feat:{} for cat_feat in low_card_cat_feat]
index = 0
for cat_feat in low_card_cat_feat:
    for value in modalities[cat_feat]:
        one_hot_encoder[cat_feat][value] = index
        index += 1
    one_hot_encoder[cat_feat][None] = index
    index += 1
dimension = index + 1 # dimension is nb_of_modalities + 1 for the intercept
dimension

### Convert to vector

In [ ]:
def row_to_vector(
    row: Row, dimension: int, encoder: Dict[str, Dict[str, int]]
) -> Tuple[np.ndarray, int]:
    x = np.zeros(dimension)
    x[-1] = 1 # for intercept
    y = row['label']
    for feat in encoder.keys():
        value = row[feat]
        index = encoder[feat].get(value, None)
        if index != None:
            x[index] = 1
    return x, y

In [ ]:
def convert_to_vectors(
    df: DataFrame, dimension: int, encoder: Dict[str, Dict[str, int]]
) -> RDD:
    features = encoder.keys()
    return df\
        .select('label', *features).rdd\
        .map(lambda row: row_to_vector(row, dimension, encoder))

In [ ]:
convert_to_vectors(full_df, dimension, one_hot_encoder).first()

## Compute loss

In [ ]:
def sigmoid(x: float) -> float:
    return 1 / (1 + math.exp(-x))

In [ ]:
X = np.arange(-10, 10, 0.01)

In [ ]:
plot.plot(X, [sigmoid(x) for x in X])

In [ ]:
def point_predict(x: np.ndarray, model: np.ndarray) -> float:
    return sigmoid(np.dot(x, model))

In [ ]:
def point_loss(prediction: float, y: int) -> float:
    return - y * math.log(prediction) - (1-y) * math.log(1-prediction)

In [ ]:
def compute_loss(vec_label_rdd: RDD, model: np.ndarray, num_examples: int) -> float:
    sum_loss = vec_label_rdd\
        .map(lambda vec_lab: point_loss(point_predict(vec_lab[0], model), vec_lab[1]))\
        .reduce(lambda u, v: u+v)
    return sum_loss / num_examples

## Compute gradient of the loss

In [ ]:
def point_gradient(x: np.ndarray, y: int, model: np.ndarray) -> float:
    p = sigmoid(np.dot(x, model))
    return (p - y) *  x

In [ ]:
def compute_gradient(vec_label_rdd: RDD, model: np.ndarray, num_examples: int) -> np.ndarray:
    sum_gradient = vec_label_rdd\
        .map(lambda vec_lab: point_gradient(vec_lab[0], vec_lab[1], model))\
        .reduce(lambda u, v: u+v)
    return sum_gradient / num_examples

## Check gradient with finite differences

In [ ]:
def point_gradient_fd(x, y, model, h=0.001):
    dimension = len(x)
    gradient = np.zeros(dimension)
    for i in range(0, dimension):
        delta = np.zeros(dimension)
        delta[i] = h
        
        loss_up = point_loss(point_predict(x, model+delta), y)
        loss_down = point_loss(point_predict(x, model-delta), y)
        
        gradient[i] = (loss_up - loss_down) / (2*h)
    return gradient

In [ ]:
model = np.random.uniform(-1.0, 1.0, size=dimension)
x = np.random.uniform(-1.0, 1.0, size=dimension)
y = 0

In [ ]:
np.sum(np.abs(point_gradient_fd(x, y, model) - point_gradient(x, y, model)))

## Distributed Gradient Descent

In [ ]:
def logit(x: float) -> float:
    return math.log( x / (1-x) )

In [ ]:
np.sum(np.abs([x - logit(sigmoid(x)) for x in X]))

In [ ]:
def smart_init(dimension: int, avg_label: float) -> np.ndarray:
    init_model = np.zeros(dimension)
    init_model[-1] = logit(avg_label)
    return init_model

In [ ]:
def train(
    training_set: DataFrame,
    dimension: int,
    encoder: Dict[str, Dict[str, int]],
    nb_iter: int,
    lr: float
) -> Tuple[np.ndarray, float]:
    num_examples, avg_label = training_set.agg(
        sf.count('*').alias('num_examples'),
        sf.mean('label').alias('avg_label')
    ).collect()[0]
    print(f'Num examples: {num_examples}, average label: {avg_label}')
    model = smart_init(dimension, avg_label)
    vector_label_rdd = convert_to_vectors(training_set, dimension, encoder).persist()
    for it in range(0, nb_iter):
        loss = compute_loss(vector_label_rdd, model, num_examples)
        print(f'Loss at step {it}: {loss}')
        gradient = compute_gradient(vector_label_rdd, model, num_examples)
        model -= lr * gradient
    final_loss = compute_loss(vector_label_rdd, model, num_examples)
    print(f'Loss at step {nb_iter}: {final_loss}')
    return model, final_loss

In [ ]:
model, loss = train(full_df, dimension, one_hot_encoder, 10, 1)

In [ ]:
print(f'intercept -> {model[-1]}')
for dim in one_hot_encoder.keys():
    for mod, index in one_hot_encoder[dim].items():
        print(f'{dim}={mod} -> {model[index]}')

In [ ]:
ss.stop()

### More questions

* use sparse vectors
* add feature hashing
* merge loss and gradient computation in one function
* use lbfgs from scipy